# Statistical Learning - Pryecto Final


### Realizado por Yurandir Santa Maria Santana Sandoval

El proyecto consiste en hacer clasificación binaria  para determinar si una persona sobrevive (y=1) o no (y=0) del hundimiento del Titanic.

En este proyecto se busca crear un modelo con un nivel de exactitud de al menos el 80%.

El proyecto está dividido en 2 partes de manera similar a como se divide o se trabaja un proyecto de ML real, es decir.

### Entrenamiento,selección y validación.

En esta parte se busca realizar el entrenamiento de los modelos  y todo lo que esto conlleva:

* Selección y transformación de features (feature eng.) utilizando características generales, no específicas como identificadores o nombres.

* Realizar las validaciones correspondientes (usando métricas de evaluación) y selección de hyper-parámetros y modelos.

### Inferencia, predicción y despliegue de modelos

En ML una vez entrenado el modelo (o modelos) estos son usados para predicción (etapa de inferencia) sobre nuevas observaciones, comúnmente integrandolos a una aplicación de software (por ejemplo, mobile o web) a través de un proceso conocido como “despliegue o deployment de ML”.

## Importación de librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics as mt
import tensorflow as tf
import random
from google.colab import drive 
from sklearn.model_selection import train_test_split

In [ ]:
import sklearn.metrics as metrics
import joblib

## Base de Datos

In [ ]:
# Autorización para Google Drive donde está cargado el archivo

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
DatabaseTitanic = pd.read_csv("/content/gdrive/My Drive/data_titanic_proyecto.csv")
DatabaseTitanic.head()

,PassengerId,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,passenger_class,passenger_sex,passenger_survived
0,1,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,S,Lower,M,N
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,C,Upper,F,Y
2,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Lower,F,Y
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,S,Upper,F,Y
4,5,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,S,Lower,M,N


In [ ]:
# Creación de atributo Passenger_survived_code para facilitar los cálculos de los modelos
# Passenger Survived = N = 0.0
# Passenger_Survived = Y = 1.0

DatabaseTitanic["Passenger_survived_code"] = (DatabaseTitanic["passenger_survived"].values == 'Y').astype(np.float32)

In [ ]:
# Completar los valores faltantes de Age con la mediana de los datos disponibles
DatabaseTitanic["Age"] = DatabaseTitanic["Age"].fillna(DatabaseTitanic["Age"].median());
DatabaseTitanic["Age"].isnull().sum()

0

In [ ]:
DatabaseTitanic.head()

,PassengerId,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,passenger_class,passenger_sex,passenger_survived,Passenger_survived_code
0,1,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,S,Lower,M,N,0.0
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,C,Upper,F,Y,1.0
2,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Lower,F,Y,1.0
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,S,Upper,F,Y,1.0
4,5,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,S,Lower,M,N,0.0


## Parte 1: Entrenamiento y validación y selección

### Train-val-test split 

El primer paso es separar los datos en entrenamiento, validación y pruebas.

Alineado a los proyectos previos se extraerá el 20% para datos de pruebas, y del 80% restante se extraerá un nuevo segmento del 20% para datos de validación.


In [ ]:
# Adaptación de la base de datos para la segmentación (inclusión solo de datos numéricos):

Fields = ["PassengerId", "Age", "SibSp", "Parch", "Fare"]

Y = DatabaseTitanic["Passenger_survived_code"].values
X = DatabaseTitanic[Fields].values

In [ ]:
# Eligiendo un número aleatorio para asignar al random_state:

Shuffle = random.randint(0,100)

# Segmentación de datos de entrenamiento:

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = Shuffle)

# Segmentación de datos de validación; esta segmentación se realiza partiendo de los datos de entrenamiento resultantes de la segmentación anterior:

X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.2, random_state = Shuffle)

Shuffle

24

### Árbol de decisión con SK Learn

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

In [ ]:
tree_model = tree.DecisionTreeClassifier(max_depth = 2)

# Entrenamiento del modelo

tree_model.fit(X_train, Y_train)

DecisionTreeClassifier(max_depth=2)

In [ ]:
y_pred = tree_model.predict(X_val)

**Métricas de evaluación**

In [ ]:
print("Accuracy Score:",round(metrics.accuracy_score(Y_val, y_pred),2))
print("Error:",round((metrics.mean_squared_error(Y_val, y_pred)),2))
print("Precision Score:",round((metrics.precision_score(Y_val, y_pred)),2))
print("Recall:",round((metrics.f1_score(Y_val, y_pred, average='weighted')),2))
print("F1 Score:",round((metrics.f1_score(Y_val, y_pred)),2))

Accuracy Score: 0.71
Error: 0.29
Precision Score: 0.76
Recall: 0.69
F1 Score: 0.52


## SVM con sklearn

In [ ]:
from sklearn import svm

In [ ]:
# Creación del classifier:

clf = svm.SVC(C=100, tol=0.001)

# Entrenamiento del modelo

clf.fit(X_train, Y_train)

SVC(C=100)

In [ ]:
# Predicción del modelo

y_pred = clf.predict(X_val)

**Métricas de evaluación**

In [ ]:
print("Accuracy Score:",round(metrics.accuracy_score(Y_val, y_pred),2))
print("Error:",round((metrics.mean_squared_error(Y_val, y_pred)),2))
print("Precision Score:",round((metrics.precision_score(Y_val, y_pred)),2))
print("Recall:",round((metrics.f1_score(Y_val, y_pred, average='weighted')),2))
print("F1 Score:",round((metrics.f1_score(Y_val, y_pred)),2))

Accuracy Score: 0.69
Error: 0.31
Precision Score: 0.7
Recall: 0.67
F1 Score: 0.49


## Naive bayes con numpy y/o pandas

## Regresión logística binaria en tensorflow con regularización

In [ ]:
from sklearn.preprocessing import OneHotEncoder

## Investigación

**K-fold cross validation**

Esta técnica parte de la necesidad de validar que un modelo cumplirá con el nivel de precisión necesaria o mínima cuando este sea puesto en producción y se enfrente a datos desconocidos o que puedan variar a los utilizados en la creación del modelo.

> **K-Folds Cross Validation**

> Es una de las técnicas que se considera tienen un menor nivel de sesgo. El procedimiento tiene un solo parámetro llamado K que se refiere al número de grupos en los que se dividirá una muestra de datos determinada, generando así un proceso iterativo generando variaciones entre los datos que componen los set de pruebas y de entrenamiento. 

> Es uno de los dos métodos de validación más reconocidos, dado que es simple de entender y porque generalmente da como resultado una estimación menos sesgada o menos optimista de la habilidad del modelo que otros métodos, como el utilizado para este proyecto que fue el método test/split en una propoción de 80/20.

> En este proyecto se pudo haber aplicado en orden de comparar de manera más exacta los diferentes métodos utilizados e incluso evaluar la combinación de parámetros más optima para cada experimento. 



